<a href="https://colab.research.google.com/github/pokjay/heb-squad/blob/main/finetune_alephbert_squad_new_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Aug 23 14:08:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Connect to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!git clone https://github.com/pokjay/heb-squad

fatal: destination path 'heb-squad' already exists and is not an empty directory.


In [4]:
!gzip -d /content/heb-squad/data/final/heb-train-v2.0.csv.gz
!gzip -d /content/heb-squad/data/final/heb-dev-v2.0.csv.gz

gzip: /content/heb-squad/data/final/heb-train-v2.0.csv.gz: No such file or directory
gzip: /content/heb-squad/data/final/heb-dev-v2.0.csv.gz: No such file or directory


In [5]:
pip install transformers

In [6]:
import collections
import pandas as pd
from tqdm.notebook import trange, tqdm

import torch
from torch.utils.data import DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
device

device(type='cuda')

In [8]:
BATCH_SIZE = 16

In [9]:
df = pd.read_csv('/content/heb-squad/data/final/heb-train-v2.0.csv')
val_df = pd.read_csv('/content/heb-squad/data/final/heb-dev-v2.0.csv')

In [10]:
df = df[(df.answer_start_heb > 0) & (df.answer_end_heb > 0)]
val_df = val_df[(val_df.answer_start_heb > 0) & (val_df.answer_end_heb > 0)]

Remove answer for impossible questions (The answer is actually the plausible answer)

In [11]:
df['plausible_answer'] = df['answer']
df['plausible_answer_start_heb'] = df['answer_start_heb']
df['plausible_answer_end_heb'] = df['answer_end_heb']

In [12]:
df.loc[df.is_impossible == 1, 'answer'] = ''
df.loc[df.is_impossible == 1, 'answer_start_heb'] = 0
df.loc[df.is_impossible == 1, 'answer_end_heb'] = 0

In [13]:
df.sample(3)

,id,context,question,answer,answer_start,answer_end,is_impossible,article,context_marked,answer_start_heb,answer_end_heb,plausible_answer,plausible_answer_start_heb,plausible_answer_end_heb
10170,56de4a81cffd8e1900b4b7b8,"HDI של 0.8 או יותר נחשב לייצג ""פיתוח גבוה"". זה...",מהי המדינה הגדולה ביותר שנכללה בין שבע המדינות...,רוּסִיָה,474,480,0,Human_Development_Index,"HDI של 0.8 או יותר נחשב לייצג ""פיתוח גבוה"". זה...",346,351,רוּסִיָה,346,351
23890,56f9453c9b226e1400dd12e4,קטע של רחוב מזרח 58th 40 ° 45′40.3 ″ N 73 ° 57...,מהו הקטע של רחוב מזרח 58 בין לקסינגטון לשדרות ...,דרך המעצבים,158,172,0,List_of_numbered_streets_in_Manhattan,קטע של רחוב מזרח 58th 40 ° 45′40.3 ″ N 73 ° 57...,148,159,דרך המעצבים,148,159
8339,56dde2609a695914005b964c,בין השנים 1590–1712 החזיקו ההולנדים גם באחד הי...,מה היה אחד מהכיבושים של הצי ההולנדי?,שבירת תחום ההשפעה הפורטוגזי על האוקיינוס ​​ההו...,143,224,0,Dutch_Republic,בין השנים 1590–1712 החזיקו ההולנדים גם באחד הי...,115,170,שבירת תחום ההשפעה הפורטוגזי על האוקיינוס ​​ההו...,115,170


## Transform the texts to encodings

In [14]:
train_ids = df.id.to_list()
train_contexts = df.context.to_list()
train_questions = df.question.to_list()

val_ids = val_df.id.to_list()
val_contexts = val_df.context.to_list()
val_questions = val_df.question.to_list()

In [15]:
train_answers = df.apply(lambda x: {'answer_start' : x.answer_start_heb, 'answer_end': x.answer_end_heb}, axis=1).to_list()
val_answers = val_df.apply(lambda x: {'answer_start' : x.answer_start_heb, 'answer_end': x.answer_end_heb}, axis=1).to_list()

In [16]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('onlplab/alephbert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [17]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        if answers[i]['answer_end'] > 0:
          end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        else:
          end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [18]:
train_encodings.update({'id': train_ids})
val_encodings.update({'id': val_ids})

## Create PyTorch dataset

In [19]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        # Get all encoded vals as tensors
        vals = {key: torch.tensor(val[idx]) for key, val in self.encodings.items() if key != 'id'}
        # Add the id which is a string, used to map predictions to ids later
        vals.update({key: val[idx] for key, val in self.encodings.items() if key == 'id'})
        return vals

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [20]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Import Hebrew BERT for Q&A fine tuning

In [21]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained("onlplab/alephbert-base")

Some weights of the model checkpoint at onlplab/alephbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initi

In [22]:
def load_checkpoint(path, model, optimizer):

  # Load checkpoint from Google Drive
  checkpoint = torch.load(path)

  # Load checkpoint to model and optimizer
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  checkpoint_epoch = checkpoint['epoch']

  return checkpoint_epoch

## Code to evaluate model

In [23]:
def calc_metrics(model_inf, dataloader):
  """
  Given a model and a dataset calculate the following metrics:
  - Exact Match
  - F1 Score
  - Loss
  @returns (F1, EM, Loss, EM Scores, F1 Scores)
  """

  model.to(device)
  model.eval()

  f1_scores = {}
  exact_scores = {}

  loss = 0
  total_cnt = 0

  for counter, batch in tqdm(enumerate(dataloader)):
    with torch.no_grad():

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      ids = batch['id']
      outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      loss += outputs[0]

      pred_start_positions = torch.argmax(outputs['start_logits'], axis=1)
      pred_end_positions = torch.argmax(outputs['end_logits'], axis=1)

      # Calc scores
      for i in range(len(input_ids)):

        total_cnt += 1

        # Check if we have an exact match
        if start_positions[i] == pred_start_positions[i] and end_positions[i] == pred_end_positions[i]:
          # We have an exact match, mark it
          exact_scores[ids[i]] = 1
        else:
          # No match, mark with 0 or keep previous marking
          exact_scores[ids[i]] = max(0, exact_scores.get(ids[i], 0))

        # Get the predicted answer token sequence
        pred_tokens = input_ids[i][pred_start_positions[i]:pred_end_positions[i] + 1].tolist()
        answer_tokens = input_ids[i][start_positions[i]:end_positions[i] + 1].tolist()

        # Check how many predicted tokens correspond to gold answer tokens
        common = collections.Counter(answer_tokens) & collections.Counter(pred_tokens)
        num_same = sum(common.values())

        # If none same, then F1=0 , else use the F1 formula
        if num_same == 0:
          score = 0
        else:
          precision = 1.0 * num_same / len(pred_tokens)
          recall = 1.0 * num_same / len(answer_tokens)
          score = (2 * precision * recall) / (precision + recall)
        
        # We take the max F1 score of the gold answers
        f1_scores[ids[i]] = max(score, f1_scores.get(ids[i], 0))

  f1 = sum(f1_scores.values()) / len(f1_scores)
  em = sum(exact_scores.values()) / len(exact_scores)
  epoch_avg_loss = loss / total_cnt

  return (f1, em, epoch_avg_loss, exact_scores, f1_scores)

## Train model

In [24]:
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

optim = AdamW(model.parameters(), lr=5e-5)

train_loss = []
val_loss = []

start_epoch = 0

checkpoint_path = '/content/drive/MyDrive/nlp/checkpoints/checkpoint_3_0.446_0.346_0.758_0.153'
start_epoch = load_checkpoint(checkpoint_path, model, optim) + 1

print(f'Training model from epoch={start_epoch}')

for epoch in range(start_epoch, 6):

    model.train()

    epoch_loss = 0

    for counter, batch in enumerate(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        epoch_loss += loss

        if counter % 250 == 0 and counter > 0:
            print('Epoch %d, %d/%d, Current Loss = %.4f' % (epoch, counter, len(train_loader), loss))

    avg_train_loss = epoch_loss / len(train_loader)

    print('Epoch %d, Train Loss = %.4f' % (epoch, avg_train_loss))

    f1, em, avg_val_loss, _, _ = calc_metrics(model, val_loader)

    print('Epoch %d, Val Loss = %.4f, F1 = %.4f, EM=%.4f' % (epoch, avg_val_loss, f1, em))

    train_loss.append(avg_train_loss)
    val_loss.append(avg_val_loss)

    state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'optimizer': optim.state_dict(),
        'f1': f1,
        'em': em,
        'avg_val_loss': avg_val_loss,
        'avg_train_loss': avg_train_loss
    }
    savepath=f'/content/drive/MyDrive/nlp/checkpoints/checkpoint_{epoch}_{f1:.3f}_{em:.3f}_{avg_train_loss:.3f}_{avg_val_loss:.3f}'
    torch.save(state, savepath)

Training model from epoch=4
Epoch 4, 250/7113, Current Loss = 0.4844
Epoch 4, 500/7113, Current Loss = 0.8463
Epoch 4, 750/7113, Current Loss = 0.5753
Epoch 4, 1000/7113, Current Loss = 0.4836
Epoch 4, 1250/7113, Current Loss = 0.3497
Epoch 4, 1500/7113, Current Loss = 0.3594
Epoch 4, 1750/7113, Current Loss = 1.0043
Epoch 4, 2000/7113, Current Loss = 0.6065
Epoch 4, 2250/7113, Current Loss = 0.8762
Epoch 4, 2500/7113, Current Loss = 0.3901
Epoch 4, 2750/7113, Current Loss = 0.1339
Epoch 4, 3000/7113, Current Loss = 0.7391
Epoch 4, 3250/7113, Current Loss = 0.3710
Epoch 4, 3500/7113, Current Loss = 0.6276
Epoch 4, 3750/7113, Current Loss = 0.2371
Epoch 4, 4000/7113, Current Loss = 1.1292
Epoch 4, 4250/7113, Current Loss = 0.5109
Epoch 4, 4500/7113, Current Loss = 1.1191
Epoch 4, 4750/7113, Current Loss = 0.2917
Epoch 4, 5000/7113, Current Loss = 0.4713
Epoch 4, 5250/7113, Current Loss = 0.6357
Epoch 4, 5500/7113, Current Loss = 0.4634
Epoch 4, 5750/7113, Current Loss = 0.5398
Epoch 4, 

0it [00:00, ?it/s]

Epoch 4, Val Loss = 0.1675, F1 = 0.4767, EM=0.3762
Epoch 5, 250/7113, Current Loss = 0.7076
Epoch 5, 500/7113, Current Loss = 0.3364
Epoch 5, 750/7113, Current Loss = 0.8248
Epoch 5, 1000/7113, Current Loss = 0.2090
Epoch 5, 1250/7113, Current Loss = 0.0538
Epoch 5, 1500/7113, Current Loss = 0.6386
Epoch 5, 1750/7113, Current Loss = 0.5536
Epoch 5, 2000/7113, Current Loss = 0.7554
Epoch 5, 2250/7113, Current Loss = 0.3633
Epoch 5, 2500/7113, Current Loss = 0.6058
Epoch 5, 2750/7113, Current Loss = 0.8002
Epoch 5, 3000/7113, Current Loss = 0.3535
Epoch 5, 3250/7113, Current Loss = 0.5457
Epoch 5, 3500/7113, Current Loss = 0.8453
Epoch 5, 3750/7113, Current Loss = 0.3671
Epoch 5, 4000/7113, Current Loss = 0.4875
Epoch 5, 4250/7113, Current Loss = 0.7399
Epoch 5, 4500/7113, Current Loss = 0.9238
Epoch 5, 4750/7113, Current Loss = 0.5459
Epoch 5, 5000/7113, Current Loss = 0.3841
Epoch 5, 5250/7113, Current Loss = 0.7498
Epoch 5, 5500/7113, Current Loss = 0.3475
Epoch 5, 5750/7113, Current 

0it [00:00, ?it/s]

Epoch 5, Val Loss = 0.2075, F1 = 0.3976, EM=0.3093


In [ ]:
def map_id_to_gold_answers(df):
  """
  In the SQuaD dev set there are multiple correct (gold) answers.
  This methods creates a mapping between the question id to
  its gold answers
  """
  ids_to_gold = {}
  ids_to_context = {}

  # Group by id as validation set has several gold answer
  for id, cols in df.groupby(by='id'):
    gold_answers = [x[1].context[x[1].answer_start_heb:x[1].answer_end_heb] for x in cols.iterrows()]
    context = cols.iloc[0].context

    # TODO: Normalize answers!

    ids_to_gold[id] = gold_answers
    ids_to_context[id] = context
  
  return ids_to_gold, ids_to_context

val_gold_answers, val_contexts = map_id_to_gold_answers(val_df)

In [ ]:
# model.to(device)

# predictions = {}

# for counter, batch in enumerate(val_loader):
#   with torch.no_grad():
  
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     start_positions = batch['start_positions'].to(device)
#     end_positions = batch['end_positions'].to(device)
#     ids = batch['id']
#     outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    
#     pred_start_positions = torch.argmax(outputs['start_logits'], axis=1)
#     pred_end_positions = torch.argmax(outputs['end_logits'], axis=1)

#     # print(pred_start_positions, pred_end_positions)

#     for i in range(1):
#       # context = val_contexts.get(ids[i])
#       x = tokenizer.convert_ids_to_tokens(input_ids[i])
#       # print(x[pred_start_positions[i]:pred_end_positions[i]+1])

#       # Start with the first token.
#       answer = x[pred_start_positions[i]]

#       # Select the remaining answer tokens and join them with whitespace.
#       for j in range(pred_start_positions[i] + 1, pred_end_positions[i] + 1):
          
#           # If it's a subword token, then recombine it with the previous token.
#           if x[j][0:2] == '##':
#               answer += x[j][2:]
          
#           # Otherwise, add a space then the token.
#           else:
#               answer += ' ' + x[j]
#       predictions[ids[i]] = answer

#       # start = val_encodings.token_to_chars(i, pred_start_positions[i])
#       # end = val_encodings.token_to_chars(i, pred_end_positions[i])
#       # pred_answer = context[start.start:end.end]
#       # print(pred_answer)


In [ ]:
# from tqdm.notebook import trange, tqdm

# def calc_metrics(model_inf, dataloader):

#   model.to(device)
#   model.eval()

#   # tp_start = 0
#   # tp_end = 0

#   f1_scores = {}
#   exact_scores = {}

#   loss = 0
#   total_cnt = 0

#   for counter, batch in tqdm(enumerate(dataloader)):
#     with torch.no_grad():

#       input_ids = batch['input_ids'].to(device)
#       attention_mask = batch['attention_mask'].to(device)
#       start_positions = batch['start_positions'].to(device)
#       end_positions = batch['end_positions'].to(device)
#       ids = batch['id']
#       outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
#       loss += outputs[0]

#       pred_start_positions = torch.argmax(outputs['start_logits'], axis=1)
#       pred_end_positions = torch.argmax(outputs['end_logits'], axis=1)

#       # # Array of True/False if pred is same as label
#       # comp_start = start_positions == pred_start_positions
#       # comp_end = end_positions == pred_end_positions
#       # score = sum(comp_start.cpu().numpy() & comp_end.cpu().numpy())
      
#       # # We take the max score of the gold answers
#       # exact_scores[ids[i]] = max(score, exact_scores.get(ids[i], 0))

#       # # True = 1 and False = 0 therefore we count how many True
#       # # tp_start += comp_start.count_nonzero()
#       # # tp_end += comp_end.count_nonzero()

#       # total_cnt += input_ids.size()[0]

#       # Calc scores
#       for i in range(len(input_ids)):

#         total_cnt += 1

#         # Check if we have an exact match
#         if start_positions[i] == pred_start_positions[i] and end_positions[i] == pred_end_positions[i]:
#           # We have an exact match, mark it
#           exact_scores[ids[i]] = 1
#         else:
#           # No match, mark with 0 or keep previous marking
#           exact_scores[ids[i]] = max(0, exact_scores.get(ids[i], 0))

#         # Get the predicted answer token sequence
#         pred_tokens = input_ids[i][pred_start_positions[i]:pred_end_positions[i] + 1].tolist()
#         answer_tokens = input_ids[i][start_positions[i]:end_positions[i] + 1].tolist()

#         # Check how many predicted tokens correspond to gold answer tokens
#         common = collections.Counter(answer_tokens) & collections.Counter(pred_tokens)
#         num_same = sum(common.values())

#         # If none same, then F1=0 , else use the F1 formula
#         if num_same == 0:
#           score = 0
#         else:
#           precision = 1.0 * num_same / len(pred_tokens)
#           recall = 1.0 * num_same / len(answer_tokens)
#           score = (2 * precision * recall) / (precision + recall)
        
#         # We take the max F1 score of the gold answers
#         f1_scores[ids[i]] = max(score, f1_scores.get(ids[i], 0))

#   f1 = sum(f1_scores.values()) / len(f1_scores)
#   em = sum(exact_scores.values()) / len(exact_scores)
#   epoch_avg_loss = loss / total_cnt

#   return (f1, em, epoch_avg_loss, exact_scores, f1_scores)

In [ ]:
# for i in val_loader:
#   batch = i
#   break

## Calculate F1 score on dataset

In [ ]:
def map_id_to_gold_answers(df):

  # Group by id as validation set has several gold answer
  for id, cols in df.groupby(by='id'):
    # for row in zip(cols.answer_start_heb, cols.answer_end_heb):
      # print(row.answer_start_heb, row.answer_end_heb)
    gold_answers = [x[1].context[x[1].answer_start_heb:x[1].answer_end_heb] for x in cols.iterrows()]
    #gold_answers = [x for x in zip(cols.answer_start_heb, cols.answer_end_heb)]
    print(id, gold_answers)
    if i == 10:
      break
    print()
    i+=1

37. 5727d1953acd2414000ded  ['צחיח למחצה']

38. 5727d1953acd2414000ded38  ['לוטון']

38. 573459cbacc1501500babe38  ["צ'סטרפילד קאונטי"]

56  ['קרדינל', '23:00', '1973']

56 dec8773277331400b4d730  ['פיטר פילז']

56 דצמבר 1673277331400b4d70d  ['']

56 דצמבר 2483277331400b4d711  ["ג'ון מקיין"]

56 דצמבר 2483277331400b4d712  ["רודי ג'וליאני"]

56 דצמבר 2483277331400b4d713  ['הסביבה והכלכלה']

56 דצמבר 4913277331400b4d720  ['שש']

56 דצמבר 4913277331400b4d721  ["לוס אנג'לס טיימס"]
